## Spider

## 关于yield

函数如果包含关键字yield，相当于一个generator。对此，有几点需要牢记：<br>
- generator中的变量引用会始终存在
- 每次只进行到yield处，返回值，但仍然会把该程序分支执行完毕
- return的值不会被抛出

In [18]:
def gen():
        print('enter')
        yield 1
        print('next')
        yield 2
        print('next again')

In [19]:
g=gen()

In [20]:
# 生成第一个yield值，此处yield被销毁
next(g)

enter


1

In [21]:
# 生成第二个yield值，此处yield被销毁
next(g)

next


2

In [22]:
next(g)

next again


StopIteration: 

In [23]:
def fib(max):
    a=0
    while a<max:
        yield a
        a=a+1  # 此while程序分支仍然会被执行，但是如果是return a，这里将不会被执行。
    return 'Done'

In [24]:
for x in fib(5):
    print(x)

0
1
2
3
4


In [25]:
def parse(dict):
    for i in dict:
        yield i
    for j in dict:
        yield j*j

In [26]:
d=[1,2,3]
for x in parse(d):
    print(x)

1
2
3
1
4
9


## 分析Spider实例

In [30]:
# -*- coding: utf-8 -*-

import scrapy

class BooksSpider(scrapy.Spider):
    name = "books" # spider唯一的标识

    start_urls=['http://books.toscrape.com/'] # 爬虫起始点

    def parse(self, response):
        
        # -解析Responese处，并生成相应的代谢产物Items-
        for book in response.css('article.product_pod'):
            name=book.xpath('./h3/a/@title').extract_first()
            price=book.css('p.price_color::text').extract_first()
            yield {
                'name':name,
                'price':price,
            }
            
        # -生成下一个url的链接，并产生下一个Request-
        next_url=response.css('ul.pager li.next a::attr(href)').extract_first()
        if next_url:
            next_url=response.urljoin(next_url)
            yield scrapy.Request(next_url,callback=self.parse)

从以上的实例中，我们主要分析parse函数，parse函数含有yield关键词，说明该函数是一个generator，那么这个generator将会生成一个怎么样的序列？<br>

首先，传入一个response对象，将会生成一个完整的for循环的序列，这个序列中包含这个response所有解析出来的Items（name、price），最后，这个generator将会生成下一个url链接，并向engine提交下一次Request请求。